In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "dlyfrzthw"

In [4]:
#Variable 
variable85="rcp85_"+var+"_seasonal"
variable45="rcp45_"+var+"_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_dlyfrzthw_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_dlyfrzthw_

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    #print(tmp1)
    i = i+1
    listds.append(tmp1)

0
1
2
3


## Loop to create all 4 dataframes

In [6]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df45f.columns = [season + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [7]:
#df45list


In [8]:
dfw45 = df45list[0].reset_index()
dfsp45 = df45list[1].reset_index()
dfsu45 = df45list[2].reset_index()
dff45 = df45list[3].reset_index()

mkey = ["lat","lon"]
dfse45 = dfw45.merge(dfsp45, on=mkey).merge(dfsu45, on=mkey).merge(dff45, on=mkey)
dfse45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_seasonal.feather")

In [9]:
dfse452 = dfse45.drop(columns=["winter_hist_rcp45_p10","winter_hist_rcp45_p50", "winter_hist_rcp45_p90", 
                               "spring_hist_rcp45_p10", "spring_hist_rcp45_p50", "spring_hist_rcp45_p90", 
                               "summer_hist_rcp45_p10","summer_hist_rcp45_p50", "summer_hist_rcp45_p90", 
                               "fall_hist_rcp45_p10","fall_hist_rcp45_p50", "fall_hist_rcp45_p90"])
dfse452


,lat,lon,winter_t2050_rcp45_p10,winter_t2080_rcp45_p10,winter_t2050_rcp45_p50,winter_t2080_rcp45_p50,winter_t2050_rcp45_p90,winter_t2080_rcp45_p90,spring_t2050_rcp45_p10,spring_t2080_rcp45_p10,...,summer_t2050_rcp45_p50,summer_t2080_rcp45_p50,summer_t2050_rcp45_p90,summer_t2080_rcp45_p90,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90
0,40.041039,-89.045212,37.50,35.79,45.53,44.90,54.53,52.59,8.23,7.07,...,0.00,0.00,0.00,0.00,6.20,4.83,10.43,9.57,16.10,15.53
1,40.041039,-88.961884,37.70,36.28,45.63,45.03,54.70,52.76,8.53,7.10,...,0.00,0.00,0.00,0.00,6.10,4.80,10.53,9.53,16.20,15.63
2,40.041039,-88.878548,37.73,36.31,45.73,44.97,54.57,52.79,8.53,7.13,...,0.00,0.00,0.00,0.00,5.93,4.83,10.53,9.30,16.03,15.53
3,40.041039,-88.795219,37.37,35.52,45.27,44.69,54.03,52.10,8.40,7.00,...,0.00,0.00,0.00,0.00,5.70,4.80,10.27,8.97,15.70,15.13
4,40.041039,-88.711891,37.17,35.55,45.17,44.48,53.77,51.93,8.20,6.87,...,0.00,0.00,0.00,0.00,5.60,4.57,10.27,8.67,15.57,14.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.17,0.10,3.60,4.86,1.60,0.53,...,18.13,14.93,28.60,24.57,14.17,15.93,21.93,21.83,31.73,33.27
76202,66.623306,-61.546310,0.00,0.00,0.10,0.03,3.33,4.69,0.90,0.17,...,19.60,16.10,30.90,26.70,12.67,14.57,19.80,20.17,29.73,30.27
76203,66.623306,-61.462978,0.00,0.00,0.13,0.03,3.53,4.83,1.23,0.47,...,18.73,16.10,30.40,25.73,13.63,15.07,21.00,21.43,31.13,31.77
76204,66.623306,-61.379650,0.00,0.00,0.13,0.03,3.50,4.93,1.43,0.50,...,18.57,16.57,30.17,26.00,13.97,15.53,21.57,21.70,31.20,32.40


In [10]:
dfsew = dfse45[["winter_hist_rcp45_p50"]]
dfsesp = dfse45[["spring_hist_rcp45_p50"]]
dfsesu = dfse45[["summer_hist_rcp45_p50"]]
dfsefa = dfse45[["fall_hist_rcp45_p50"]]

# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [11]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 4 dataframes

In [12]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df85f.columns = [season + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [13]:
df85list


[                      winter_hist_rcp85_p10  winter_t2050_rcp85_p10  \
 lat       lon                                                         
 40.041039 -89.045212                  40.60                   35.07   
           -88.961884                  40.60                   35.43   
           -88.878548                  40.43                   35.43   
           -88.795219                  39.97                   35.07   
           -88.711891                  39.73                   35.03   
 ...                                     ...                     ...   
 66.623306 -61.629639                   0.00                    0.00   
           -61.546310                   0.00                    0.00   
           -61.462978                   0.00                    0.00   
           -61.379650                   0.00                    0.00   
           -61.296322                   0.00                    0.00   
 
                       winter_t2080_rcp85_p10  winter_hist_rcp

In [14]:
dfw85 = df85list[0].reset_index()
dfsp85 = df85list[1].reset_index()
dfsu85 = df85list[2].reset_index()
dff85 = df85list[3].reset_index()
mkey = ["lat","lon"]
dfse85 = dfw85.merge(dfsp85, on=mkey).merge(dfsu85, on=mkey).merge(dff85, on=mkey)
dfse85

,lat,lon,winter_hist_rcp85_p10,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_hist_rcp85_p50,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_hist_rcp85_p90,winter_t2050_rcp85_p90,...,summer_t2080_rcp85_p90,fall_hist_rcp85_p10,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_hist_rcp85_p50,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_hist_rcp85_p90,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,40.60,35.07,29.41,50.73,44.03,38.59,60.07,52.90,...,0.00,11.30,3.50,1.07,17.23,8.80,4.53,23.00,13.63,10.03
1,40.041039,-88.961884,40.60,35.43,29.72,50.80,44.33,39.00,60.07,53.27,...,0.00,11.30,3.63,1.10,17.13,8.83,4.57,22.83,13.73,10.07
2,40.041039,-88.878548,40.43,35.43,29.76,50.73,44.33,39.28,60.03,53.03,...,0.00,11.17,3.57,1.10,17.10,8.53,4.53,22.87,13.67,10.13
3,40.041039,-88.795219,39.97,35.07,29.66,50.07,43.97,38.93,59.30,52.40,...,0.00,10.93,3.47,1.03,16.67,8.40,4.53,22.57,13.50,9.97
4,40.041039,-88.711891,39.73,35.03,29.45,49.90,43.70,38.72,58.93,52.37,...,0.00,10.77,3.23,1.00,16.10,8.07,4.33,22.33,13.27,9.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.33,0.86,0.93,6.93,...,16.67,13.97,16.97,18.50,19.80,23.90,27.47,27.60,35.00,43.73
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.10,0.52,0.63,6.50,...,17.67,12.83,14.67,17.47,18.47,21.90,25.43,26.13,31.97,41.30
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.23,0.76,0.73,6.77,...,17.23,13.50,16.10,18.27,19.43,23.07,26.73,27.00,33.93,43.33
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.23,0.72,0.80,6.97,...,17.27,13.93,16.63,18.60,19.77,23.47,26.90,27.73,34.43,44.10


In [15]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_seasonal.feather")

# Merge df45 and 85

# Extract hist

In [16]:
dfse852 = dfse85.drop(columns=["winter_hist_rcp85_p10","winter_hist_rcp85_p50", "winter_hist_rcp85_p90", 
                               "spring_hist_rcp85_p10", "spring_hist_rcp85_p50", "spring_hist_rcp85_p90", 
                               "summer_hist_rcp85_p10","summer_hist_rcp85_p50", "summer_hist_rcp85_p90", 
                               "fall_hist_rcp85_p10","fall_hist_rcp85_p50", "fall_hist_rcp85_p90"])
dfse852


,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,summer_t2050_rcp85_p50,summer_t2080_rcp85_p50,summer_t2050_rcp85_p90,summer_t2080_rcp85_p90,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,35.07,29.41,44.03,38.59,52.90,47.28,6.80,3.40,...,0.00,0.00,0.00,0.00,3.50,1.07,8.80,4.53,13.63,10.03
1,40.041039,-88.961884,35.43,29.72,44.33,39.00,53.27,47.52,6.97,3.47,...,0.00,0.00,0.00,0.00,3.63,1.10,8.83,4.57,13.73,10.07
2,40.041039,-88.878548,35.43,29.76,44.33,39.28,53.03,47.66,6.90,3.57,...,0.00,0.00,0.00,0.00,3.57,1.10,8.53,4.53,13.67,10.13
3,40.041039,-88.795219,35.07,29.66,43.97,38.93,52.40,47.55,6.73,3.47,...,0.00,0.00,0.00,0.00,3.47,1.03,8.40,4.53,13.50,9.97
4,40.041039,-88.711891,35.03,29.45,43.70,38.72,52.37,47.17,6.77,3.40,...,0.00,0.00,0.00,0.00,3.23,1.00,8.07,4.33,13.27,9.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.33,0.86,6.93,18.79,0.93,0.50,...,16.57,9.47,24.13,16.67,16.97,18.50,23.90,27.47,35.00,43.73
76202,66.623306,-61.546310,0.00,0.00,0.10,0.52,6.50,17.90,0.33,0.13,...,17.63,10.10,26.20,17.67,14.67,17.47,21.90,25.43,31.97,41.30
76203,66.623306,-61.462978,0.00,0.00,0.23,0.76,6.77,18.31,0.73,0.43,...,17.13,10.17,25.30,17.23,16.10,18.27,23.07,26.73,33.93,43.33
76204,66.623306,-61.379650,0.00,0.00,0.23,0.72,6.97,18.28,0.90,0.50,...,17.13,10.00,25.30,17.27,16.63,18.60,23.47,26.90,34.43,44.10


## Getting historic mean for each season

In [17]:
### Function
def mean_season(season, dfse45, dfse85):
    dfw45 = dfse45[["lat", "lon", season+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", season+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[season+"_hist_p50"] = round((dfsew[season+"_hist_rcp45_p50"]+ dfsew[season+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", season+"_hist_p50"]]
    return (dftgwh3)


### Winter historic mean

In [18]:
dftgwh = mean_season("winter", dfse45, dfse85)
dftgwh

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,50.38
1,40.041039,-88.961884,50.55
2,40.041039,-88.878548,50.38
3,40.041039,-88.795219,49.67
4,40.041039,-88.711891,49.52
...,...,...,...
76201,66.623306,-61.629639,0.00
76202,66.623306,-61.546310,0.00
76203,66.623306,-61.462978,0.00
76204,66.623306,-61.379650,0.00


### Spring historic mean

In [19]:
dftsph = mean_season("spring", dfse45, dfse85)
dftsph

,lat,lon,spring_hist_p50
0,40.041039,-89.045212,20.62
1,40.041039,-88.961884,20.58
2,40.041039,-88.878548,20.58
3,40.041039,-88.795219,20.62
4,40.041039,-88.711891,20.50
...,...,...,...
76201,66.623306,-61.629639,9.98
76202,66.623306,-61.546310,8.55
76203,66.623306,-61.462978,9.32
76204,66.623306,-61.379650,9.86


### Summer historic mean

In [20]:
dftsuh = mean_season("summer", dfse45, dfse85)
dftsuh

,lat,lon,summer_hist_p50
0,40.041039,-89.045212,0.00
1,40.041039,-88.961884,0.00
2,40.041039,-88.878548,0.00
3,40.041039,-88.795219,0.00
4,40.041039,-88.711891,0.00
...,...,...,...
76201,66.623306,-61.629639,25.42
76202,66.623306,-61.546310,27.18
76203,66.623306,-61.462978,25.87
76204,66.623306,-61.379650,25.74


### Fall historic mean

In [21]:
dftfah = mean_season("fall", dfse45, dfse85)
dftfah

,lat,lon,fall_hist_p50
0,40.041039,-89.045212,17.12
1,40.041039,-88.961884,17.08
2,40.041039,-88.878548,17.02
3,40.041039,-88.795219,16.55
4,40.041039,-88.711891,16.12
...,...,...,...
76201,66.623306,-61.629639,19.58
76202,66.623306,-61.546310,18.20
76203,66.623306,-61.462978,19.23
76204,66.623306,-61.379650,19.45


# Merge means with the 45 and 85 DF

In [22]:
dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [23]:
dftgseall

,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90,fall_hist_p50,summer_hist_p50,spring_hist_p50,winter_hist_p50
0,40.041039,-89.045212,35.07,29.41,44.03,38.59,52.90,47.28,6.80,3.40,...,6.20,4.83,10.43,9.57,16.10,15.53,17.12,0.00,20.62,50.38
1,40.041039,-88.961884,35.43,29.72,44.33,39.00,53.27,47.52,6.97,3.47,...,6.10,4.80,10.53,9.53,16.20,15.63,17.08,0.00,20.58,50.55
2,40.041039,-88.878548,35.43,29.76,44.33,39.28,53.03,47.66,6.90,3.57,...,5.93,4.83,10.53,9.30,16.03,15.53,17.02,0.00,20.58,50.38
3,40.041039,-88.795219,35.07,29.66,43.97,38.93,52.40,47.55,6.73,3.47,...,5.70,4.80,10.27,8.97,15.70,15.13,16.55,0.00,20.62,49.67
4,40.041039,-88.711891,35.03,29.45,43.70,38.72,52.37,47.17,6.77,3.40,...,5.60,4.57,10.27,8.67,15.57,14.90,16.12,0.00,20.50,49.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.33,0.86,6.93,18.79,0.93,0.50,...,14.17,15.93,21.93,21.83,31.73,33.27,19.58,25.42,9.98,0.00
76202,66.623306,-61.546310,0.00,0.00,0.10,0.52,6.50,17.90,0.33,0.13,...,12.67,14.57,19.80,20.17,29.73,30.27,18.20,27.18,8.55,0.00
76203,66.623306,-61.462978,0.00,0.00,0.23,0.76,6.77,18.31,0.73,0.43,...,13.63,15.07,21.00,21.43,31.13,31.77,19.23,25.87,9.32,0.00
76204,66.623306,-61.379650,0.00,0.00,0.23,0.72,6.97,18.28,0.90,0.50,...,13.97,15.53,21.57,21.70,31.20,32.40,19.45,25.74,9.86,0.00


In [24]:
dftgseall.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_seasonal.feather")

In [25]:
#dftgseall = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [26]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [27]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [28]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [29]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_seasonal.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_seasonal.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_dlyfrzthw_seasonal.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
